In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


22/11/06 08:50:05 WARN Utils: Your hostname, DESKTOP-TVDS9T6 resolves to a loopback address: 127.0.1.1; using 172.29.38.221 instead (on interface eth0)
22/11/06 08:50:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/06 08:50:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
import dask.dataframe as dd
import pyspark.pandas as ps

X = pd.DataFrame(zip(range(100), range(100)), columns=['x','y'])
X_dask = dd.from_pandas(X, npartitions=1)
X_spark = spark.createDataFrame(X)

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [10]:
# from multipledispatch import dispatch
# import pandas as pd

# def distributable(sig):
#     def decorator(func):
#         # Register the distributed implementations
#         dispatch((dd.Series, dd.DataFrame))(func)

#         # Return the base implementation as well
#         return dispatch(sig)(func)

#     return decorator

# @distributable((pd.Series, pd.DataFrame))
# def test(df):
#     return df // 2

# @dispatch((dd.Series, dd.DataFrame))
# def test(df):
#     return df * 2

test(X_dask['x'])

Dask Series Structure:
npartitions=1
0     int64
99      ...
Name: x, dtype: int64
Dask Name: floordiv, 3 graph layers

In [12]:
global_namespace['test'].funcs

{(pandas.core.series.Series,): <function __main__.test(df)>,
 (pandas.core.frame.DataFrame,): <function __main__.test(df)>,
 (int,): <function __main__.Test.test(df)>,
 (dask.dataframe.core.Series,): <function __main__.test(df)>,
 (dask.dataframe.core.DataFrame,): <function __main__.test(df)>}

In [9]:
from dfender import distribute
from multipledispatch import dispatch
from multipledispatch.core import global_namespace

def distributable(sig):
    def decorator(func):
        # Register the distributed implementations
        dispatch((dd.Series, dd.DataFrame))(func)

        # Return the base implementation as well
        return dispatch(sig)(func)

    return decorator

class Test:
    @distributable((pd.Series, pd.DataFrame))
    def test(df):
        return df // 4

    @distributable(int)
    def test(df):
        return df == 0

    # @distribute
    # def predict(self, X: pd.Series) -> pd.Series:
    #     return X == 2
    #     # return (pd.to_datetime('2000-01-01').to_period('M') + X).dt.to_timestamp()

    # @distribute
    # def rowmax(self, X: pd.DataFrame):
    #     return X.max(axis=1)
    # @dispatch(pd.DataFrame)
    # def predict(self, X: pd.DataFrame) -> pd.Series:
    #     return X * 2

    # @dispatch(dd.DataFrame)
    # def predict(self, X: dd.DataFrame) -> dd.Series:
    #     return X / 2


est = Test()
# est.predict(X['x'])
# est.predict(X_dask['x']).compute()
# est.predict(X_spark.select(['x']))
# est.rowmax(X)
# est.rowmax(X_)
# Test.test.funcs
est.test.funcs

{(pandas.core.series.Series,): <function __main__.Test.test(df)>,
 (pandas.core.frame.DataFrame,): <function __main__.Test.test(df)>,
 (int,): <function __main__.Test.test(df)>,
 (dask.dataframe.core.Series,): <function __main__.Test.test(df)>,
 (dask.dataframe.core.DataFrame,): <function __main__.Test.test(df)>}

In [7]:
(pd.to_datetime('2000-01-01').to_period('M') + pd.DataFrame([], columns=['x'],dtype='int64')['x']).dt.to_timestamp()

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
spark.createDataFrame(pd.DataFrame([0])).schema[0].dataType.simpleString()

'bigint'

In [ ]:
spark.createDataFrame(pd.DataFrame([0])).schema[0].name

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


'0'

In [ ]:
import pyspark
pyspark.sql.DataFrame(X)

TypeError: __init__() missing 1 required positional argument: 'sql_ctx'

In [ ]:
def native_meta(df: pd.DataFrame):
    # Parse out the spark udf signature from the derived datatypes
    df_spark = spark.createDataFrame(df)
    dtypes = {
        element.name : element.dataType.simpleString() for element in df_spark.schema
    }
    return ', '.join([f"{col} {dtype}" for col, dtype in dtypes.items()])

native_meta(X)

'x bigint, y bigint'

In [ ]:
dtypes = X_spark.pandas_api().dtypes.to_dict()
meta = pd.DataFrame({col : pd.Series(dtype=dtype) for col, dtype in dtypes.items()})

x    int64
y    int64
dtype: object

In [ ]:
from dfender.materializers import materializer

materializer

Dfender Materializer Registry

Implementations: 
dask: [<class 'dask.dataframe.core.Series'>, <class 'dask.dataframe.core.DataFrame'>]
spark: [<class 'pyspark.pandas.series.Series'>, <class 'pyspark.pandas.frame.DataFrame'>]